<a href="https://colab.research.google.com/github/myhub889/51bitqunt/blob/master/Build_A_Stk_Prediction_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
pip install quandl


In [0]:
# this program predicts stock prices by using machine learning models # install the dependencies
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [43]:
# get the stock data
df=quandl.get("WIKI/MU", authtoken="jpyN_Jd_U-pAtUZPV71n")
# take a look at the data
print(df)

             Open   High    Low  ...   Adj. Low  Adj. Close  Adj. Volume
Date                             ...                                    
1989-05-16  24.50  24.63  23.38  ...   2.263457    2.286692   13134800.0
1989-05-17  23.88  24.87  23.62  ...   2.284888    2.345831    8472000.0
1989-05-18  24.50  24.50  24.00  ...   2.324353    2.335974    4792800.0
1989-05-19  24.25  24.87  24.12  ...   2.335974    2.335974    6996800.0
1989-05-22  24.38  25.37  24.25  ...   2.339775    2.424682    9036800.0
...           ...    ...    ...  ...        ...         ...          ...
2018-03-21  61.35  61.72  60.60  ...  60.600000   61.070000   50063889.0
2018-03-22  60.07  60.30  58.11  ...  58.110000   58.970000   66693263.0
2018-03-23  57.69  58.80  53.68  ...  53.680000   54.210000  117093861.0
2018-03-26  56.26  56.71  52.72  ...  52.720000   55.560000   69445328.0
2018-03-27  55.06  55.47  51.68  ...  51.680000   52.400000   67229050.0

[7273 rows x 12 columns]


In [44]:
# get the adjusted close price
df=df[["Adj. Close"]]
# take a look at the new data
print(df.tail())

            Adj. Close
Date                  
2018-03-21       61.07
2018-03-22       58.97
2018-03-23       54.21
2018-03-26       55.56
2018-03-27       52.40


In [45]:
# A variable for predicting n days out into the future
forecast_out=33
# create another column(the target or dependent variable)shifted "n" units up
df["prediction"]=df[["Adj. Close"]].shift(-forecast_out)
# print the new data set
print(df.tail())

            Adj. Close  prediction
Date                              
2018-03-21       61.07         NaN
2018-03-22       58.97         NaN
2018-03-23       54.21         NaN
2018-03-26       55.56         NaN
2018-03-27       52.40         NaN


In [46]:
# create the indenpent data set (x) # convert the dataframe to a numpy array
x=np.array(df.drop(["prediction"],1))
# remove the last n rows
x=x[:-forecast_out]
print(x)

[[ 2.28669229]
 [ 2.34583088]
 [ 2.33597445]
 ...
 [39.62      ]
 [43.88      ]
 [42.01      ]]


In [47]:
# create the dependent data set(y)# convert the dataframe to a numpy array(all of the values including the NAN's)
y=np.array(df["prediction"])
# get all of the values except the last n rows 
y=y[:-forecast_out]
print(y)

[ 1.78401424  1.77415781  1.76430138 ... 54.21       55.56
 52.4       ]


In [0]:
# split the data into 80% training and 20% testing
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [49]:
# create and train the support vector machine(regressor)
svr_rbf=SVR(kernel="rbf",C=1e3,gamma=0.1)
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [50]:
# testing model:score returns the coefficient of determination R^2 of the prediction.the best possible score is 1.0
svm_confidence=svr_rbf.score(x_test,y_test)
print("svm confidence:",svm_confidence)

svm confidence: 0.842656152909864


In [51]:
# create and train the linear regression model 
lr=LinearRegression()
# train the model
lr.fit(x_train,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [52]:
# testing model:score returns the coefficient of determination R^2 of the prediction.the best possible score is 1.0
lr_confidence=lr.score(x_test,y_test)
print("lr confidence:",lr_confidence)

lr confidence: 0.8350750970109206


In [53]:
# set x_forecast equal to the last 30 rows of the original data set from Adj.close column
x_forecast=np.array(df.drop(["prediction"],1))[-forecast_out:]
print(x_forecast)

[[40.  ]
 [40.35]
 [42.19]
 [42.63]
 [43.45]
 [43.5 ]
 [44.21]
 [44.89]
 [44.54]
 [44.25]
 [46.53]
 [47.98]
 [48.58]
 [48.81]
 [47.62]
 [49.11]
 [52.03]
 [53.74]
 [53.97]
 [55.22]
 [54.59]
 [59.37]
 [59.43]
 [59.78]
 [58.84]
 [60.58]
 [60.14]
 [61.15]
 [61.07]
 [58.97]
 [54.21]
 [55.56]
 [52.4 ]]


In [54]:
# print the linear regression model predictions for the next n days 
lr_predictions=lr.predict(x_forecast)
print(lr_predictions)
# print support vector regression model predictions for the next n days 
svm_predictions=svr_rbf.predict(x_forecast)
print(svm_predictions)

[38.7927762  39.12224925 40.85433611 41.2685308  42.04043907 42.08750665
 42.75586626 43.39598532 43.06651227 42.79352032 44.93980187 46.30476163
 46.86957256 47.08608342 45.96587507 47.36848889 50.11723544 51.7269466
 51.94345746 53.12014691 52.52709542 57.02675587 57.08323696 57.41271001
 56.52783954 58.16579125 57.75159657 58.70236164 58.62705352 56.65021525
 52.16938183 53.44020643 50.46553551]
[39.13341652 39.5562972  41.92498643 42.511667   42.74867245 42.71094728
 41.59105007 40.26931533 40.89109205 41.5063996  42.18199041 43.12824471
 42.27926274 42.20116505 43.5404544  42.52314253 60.23977327 65.0234288
 65.61308422 66.41665261 66.58536919 70.76085828 70.97243528 72.49312583
 69.4412485  77.11227889 74.48841459 79.64918717 79.40325713 69.68753479
 66.11669763 66.02312887 61.33393501]
